2-1. Nate 뉴스기사 제목 스크래핑하기 (필수)
https://news.nate.com/recent?mid=n0100
최신뉴스, 정치 , 경제, 사회, 세계, IT/과학 
6개의 섹션의 뉴스를 출력하는 함수를 생성하여 스크래핑 하기

Image, 기사제목, 기사링크

뉴스기사의 Image를 출력 하세요 
1) Image의 절대경로와 상대 경로를 합치려면 urljoin 함수를 사용하세요.
    from urllib.parse import urljoin

2) Image 출력은 Image 클래스와 display 함수를 사용하세요.
    from IPython.display import Image, display

3) img 엘리먼트의 존재 여부를 체크하신 후에 src 속성의 이미지를 경로를 추출하기
  => Image 가 없는 뉴스도 있기 때문에 



In [19]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display

section_dict = {'최신뉴스':'n0100','정치':'n0200','경제':'n0300','사회':'n0400',\
                '세계':'n0500','IT/과학':'n0600'}
print(section_dict)

def print_news(section_name):    
    m_id = section_dict[section_name]

    url = f'https://news.nate.com/recent?mid={m_id}'
    print(url)

    req_header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }

    res = requests.get(url, headers=req_header)
    print(type(res))
    print(res.status_code)

    if res.ok:
        res.encoding = 'euc-kr'
        html = res.text
        soup = BeautifulSoup(html, 'html.parser')

        print(f'======> {url} {section_name} 뉴스 <======')   

        tags = soup.select("div.postListType.noListTitle div.mlt01")
        print(len(tags))

        for idx,div_tag in enumerate(tags,1): 
            print(f'============>> {idx}')       
            a_tag = div_tag.find('a')
            a_join_url = urljoin(url,a_tag['href'])
            print(a_join_url)

            img_tag = div_tag.select_one('span.ib img')
            if img_tag:
                photo_url = urljoin(url,img_tag['src'])
                print(photo_url)
                display(Image(url=photo_url))

            h2_tag = div_tag.select_one('span.tb h2.tit')
            title = h2_tag.text
            print(title)

    else:
        print(f'에러코드 = {res.status_code}')

print_news('경제')

{'최신뉴스': 'n0100', '정치': 'n0200', '경제': 'n0300', '사회': 'n0400', '세계': 'n0500', 'IT/과학': 'n0600'}
https://news.nate.com/recent?mid=n0300
<class 'requests.models.Response'>
200
======> https://news.nate.com/recent?mid=n0300 경제 뉴스 <======
20
============>> 1
https://news.nate.com/view/20250725n08859?mid=n0300
https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/mk/2025/07/25/news-p.v1.20250725.2423242a91a94e7293c029682ed5ab55_P1.png


LG, SK move to resume transactions
============>> 2
https://news.nate.com/view/20250725n08858?mid=n0300
https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/mw/2025/07/25/2025072509142277111_1.jpg


[아침밥] KB증권 "SK하이닉스, 실적 성장 지속…경쟁사와 차별"
============>> 3
https://news.nate.com/view/20250725n08857?mid=n0300
https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/mk/2025/07/25/news-p.v1.20250725.bbf84bb69a80435c99cd89e169311e23_P1.jpg


KB Financial achieves record net profit despite declining interest income
============>> 4
https://news.nate.com/view/20250725n08856?mid=n0300
https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/mw/2025/07/25/2025072509324755253_1.jpg


[아침밥]미래에셋 "현대제철, 아쉬운 별도 손익에도 긍정적"
============>> 5
https://news.nate.com/view/20250725n08855?mid=n0300
https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/mw/2025/07/25/2025072508191181061_1.jpg


...

JB Financial acquires KB Bukopin Finance
============>> 17
https://news.nate.com/view/20250725n08821?mid=n0300
https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/yt/2025/07/25/AKR20250725048800003_01_i.jpg


[게시판] 한국건설산업연구원, 내달 '건설산업 규제 개혁' 세미나
============>> 18
https://news.nate.com/view/20250725n08820?mid=n0300
https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/ni/2025/07/25/2507250947364660.jpg


네이버, 고양이의 날 맞아 반려묘 사진 대회 개최
============>> 19
https://news.nate.com/view/20250725n08818?mid=n0300
https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/hr/2025/07/25/news-p.v1.20250725.5e321e21f7dd4ad8a4c9288b288c822d_P1.jpg


코레일, 외국인 철도관광 활성화 위한 간담회 개최
============>> 20
https://news.nate.com/view/20250725n08816?mid=n0300
https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/fr/2025/07/25/art_697620_1753404813.jpg


'상생협력 모델 구축' 한화오션, 사외협력사 시상식


2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기 (필수)
:  하나의 웹툰의 제목(title)과 회차번호(no),회차의URL(url) 을 입력으로 받는 함수를 선언합니다. 
   def download_one_episode(title,no,url):

아래와 같이 호출합니다.
download_one_episode('일렉시드',341,'https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed')

img\일렉시드\341 디렉토리가 생성되며 , 
그 디렉토리 아래에 웹툰 image들이 다운로드 되도록 해주세요.


In [ ]:
import os
import requests
from bs4 import BeautifulSoup

def download_one_episode(title, no, url):
    """
    네이버 웹툰의 한 회차 이미지를 모두 다운로드합니다.

    :param title: 웹툰 제목 (폴더명으로 사용)
    :param no: 회차 번호 (폴더명으로 사용)
    :param url: 다운로드할 회차의 URL
    """
    # 저장할 디렉토리 경로 설정 및 생성
    # 예: img/광마회귀/160
    save_path = os.path.join('img', title, str(no))
    os.makedirs(save_path, exist_ok=True)
    print(f"'{save_path}' 디렉토리를 생성했습니다.")

    # 웹툰 페이지에 접속하기 위한 헤더 설정 (차단 방지)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Referer': url
    }

    # 웹툰 페이지의 HTML 가져오기
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # HTTP 오류가 발생하면 예외 발생
    except requests.exceptions.RequestException as e:
        print(f"오류: 웹 페이지를 가져올 수 없습니다. {e}")
        return

    # HTML 파싱하여 이미지 URL 추출
    soup = BeautifulSoup(response.text, 'html.parser')
    image_tags = soup.select('.wt_viewer img') # 웹툰 이미지를 담고 있는 태그 선택

    if not image_tags:
        print("오류: 이미지 태그를 찾을 수 없습니다. 페이지 구조가 변경되었을 수 있습니다.")
        return

    # 각 이미지 다운로드
    print("이미지 다운로드를 시작합니다...")
    for i, img_tag in enumerate(image_tags):
        img_url = img_tag.get('src')
        if not img_url:
            continue

        try:
            # 이미지 데이터 요청
            img_response = requests.get(img_url, headers=headers)
            img_response.raise_for_status()

            # 파일명 설정 (예: 001.jpg, 002.jpg, ...)
            file_name = f"{i+1:03d}.jpg"
            file_path = os.path.join(save_path, file_name)

            # 이미지 파일 저장
            with open(file_path, 'wb') as f:
                f.write(img_response.content)
            
            print(f"다운로드 완료: {file_name}")

        except requests.exceptions.RequestException as e:
            print(f"오류: '{img_url}' 이미지 다운로드에 실패했습니다. {e}")
            
    print(f"\n총 {len(image_tags)}개의 이미지를 '{save_path}'에 성공적으로 다운로드했습니다. 🎉")


# --- 함수 호출 예시 ---
if __name__ == "__main__":
    download_one_episode('광마회귀', 160, 'https://comic.naver.com/webtoon/detail?titleId=776601&no=160&week=fri')

'img\광마회귀\160' 디렉토리를 생성했습니다.
이미지 다운로드를 시작합니다...
다운로드 완료: 001.jpg
다운로드 완료: 002.jpg
다운로드 완료: 003.jpg
다운로드 완료: 004.jpg
다운로드 완료: 005.jpg
다운로드 완료: 006.jpg
다운로드 완료: 007.jpg
다운로드 완료: 008.jpg
다운로드 완료: 009.jpg
다운로드 완료: 010.jpg
다운로드 완료: 011.jpg
다운로드 완료: 012.jpg
다운로드 완료: 013.jpg
다운로드 완료: 014.jpg
다운로드 완료: 015.jpg
다운로드 완료: 016.jpg
다운로드 완료: 017.jpg
다운로드 완료: 018.jpg
다운로드 완료: 019.jpg
다운로드 완료: 020.jpg
다운로드 완료: 021.jpg
다운로드 완료: 022.jpg
다운로드 완료: 023.jpg
다운로드 완료: 024.jpg
다운로드 완료: 025.jpg
다운로드 완료: 026.jpg
다운로드 완료: 027.jpg
다운로드 완료: 028.jpg
다운로드 완료: 029.jpg
다운로드 완료: 030.jpg
다운로드 완료: 031.jpg
다운로드 완료: 032.jpg
다운로드 완료: 033.jpg
다운로드 완료: 034.jpg
다운로드 완료: 035.jpg
다운로드 완료: 036.jpg
다운로드 완료: 037.jpg
다운로드 완료: 038.jpg
다운로드 완료: 039.jpg
다운로드 완료: 040.jpg
다운로드 완료: 041.jpg
다운로드 완료: 042.jpg
다운로드 완료: 043.jpg
다운로드 완료: 044.jpg
다운로드 완료: 045.jpg
다운로드 완료: 046.jpg
다운로드 완료: 047.jpg
다운로드 완료: 048.jpg
다운로드 완료: 049.jpg
다운로드 완료: 050.jpg
다운로드 완료: 051.jpg
다운로드 완료: 052.jpg
다운로드 완료: 053.jpg
다운로드 완료: 054.jpg
다운로드 완료: 055.jpg
다운로드 완료: 056.jpg
